In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import warnings
warnings.filterwarnings("ignore")
import textwrap

In [3]:
!pip install -q accelerate tiktoken transformers_stream_generator einops optimum  auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/

Qwen 1.8B with Int4

In [ ]:
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-1_8B-Chat", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-1_8B-Chat-Int4", #"Qwen/Qwen-7B-Chat-Int4",
    device_map="auto",
    trust_remote_code=True
).eval()


model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

Qwen 14B with Int4 (using either 1.8 or 14B in one run)

In [2]:
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-14B-Chat-Int4", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-14B-Chat-Int4",
    device_map="auto",
    trust_remote_code=True
).eval()

model.safetensors.index.json:   0%|          | 0.00/82.2k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/2.04G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

In [3]:
# load the dataset
with open('corpus.txt','r',encoding="utf8") as f:
    document = f.readlines()
f.close()

labels, texts = [], []
for line in document:
    content = line.split()
    label = content[0]
    labels.append(label[-1])
    texts.append(" ".join(content[1:]))

print(len(labels), len(texts))

10000 10000


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train2, X_test2, y_train, y_test  = train_test_split(texts, labels, train_size=0.75,random_state=1234)

In [6]:
qwen_pred = []
for i, s in enumerate(X_test2):
    if (i % 100) == 0:
        print(i)
    text = s
    prompt = f"""
    What is the sentiment of the following text, which is delimited with triple backticks?

    Give your answer as a single word, either "positive" or "negative".
    Review text: '''{text}'''
    """
    response, history = model.chat(tokenizer, prompt, history=None)
    qwen_pred.append(response)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


In [7]:
qwen_pred[0:10]

['positive',
 'The sentiment of the review text is negative.',
 'positive',
 'negative',
 'positive',
 'Positive.',
 'negative',
 'Positive.',
 'The sentiment of the review text is "negative".',
 'negative']

In [8]:
sentiments = []
for s in qwen_pred:
  if 'positive' in s.lower():
    sentiments.append('2')
  else:
    sentiments.append('1')

In [ ]:
sentiments[0:10]

['2', '1', '2', '1', '2', '2', '1', '2', '1', '1']

In [9]:
from sklearn.metrics import accuracy_score

In [10]:
accuracy_score(y_test, sentiments)

0.944